In [5]:
!pip install ir_datasets
!pip install Whoosh
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=6bacc607c682f942bcf1bb368af499d9b3537de2bdd99260da55c4ac0d6b6a02
  Stored in directory: /root/.cache/pip/wheels/1a/d7/91/7ffb991df87e62355d945745035470ba2616aa3d83a250b5f9
  Created wheel for zlib-state: filename=zlib_state-0.1.6-cp310-cp310-linux_x86_64.whl size=21164 sha256=5f7cf25bbb18fc30373b8f5502e4

In [6]:
import os

import ir_datasets
import pyterrier as pt

In [7]:
if not pt.started():
  pt.init()

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [8]:
dataset = pt.get_dataset("irds:beir/msmarco/test")

In [9]:
indexer = pt.IterDictIndexer('./indices/beir_msmarco')
index_ref = indexer.index(dataset.get_corpus_iter(), fields=['text'])

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/msmarco.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/444067daf65d982533ea17ebd59501e4
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/msmarco.zip
docs_iter:   0%|                                   | 0/8841823 [00:00<?, ?doc/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/msmarco.zip: 0.0%| 0.00/1.08G [00:00<?, ?B/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/msmarco.zip: 0.0%| 81.9k/1.08G [00:00<28:22, 636kB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/msmarco.zip: 0.0%| 180k/1.08G [00:00<28:47, 626kB/s] 
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/msmarco.zip: 0.0%| 434k/1.08G [00:00<16:05, 1.12MB/s]
https://public.ukp.informatik.tu-darmstadt.

beir/msmarco/test documents:   0%|          | 0/8841823 [00:00<?, ?it/s]

07:23:57.759 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (500080) - further warnings are suppressed
07:54:17.705 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 5 empty documents


In [10]:
index = pt.IndexFactory.of(index_ref)

In [11]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")
dph = pt.BatchRetrieve(index, wmodel="DPH")
linear = bm25 + 2 * dph

In [12]:
topic = dataset.get_topics()
qrels = dataset.get_qrels()

[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]
[INFO] [starting] opening zip file
[INFO] [finished] opening zip file s]


In [13]:
from pyterrier.measures import *

In [14]:
ir_dataset = ir_datasets.load("beir/msmarco/test")
docstore = ir_dataset.docs_store()

In [16]:
for query in ir_dataset.queries_iter():
  print(query)

GenericQuery(query_id='19335', text='anthropological definition of environment')
GenericQuery(query_id='47923', text='axon terminals or synaptic knob definition')
GenericQuery(query_id='87181', text='causes of left ventricular hypertrophy')
GenericQuery(query_id='87452', text='causes of military suicide')
GenericQuery(query_id='104861', text='cost of interior concrete flooring')
GenericQuery(query_id='130510', text='definition declaratory judgment')
GenericQuery(query_id='131843', text='definition of a sigmet')
GenericQuery(query_id='146187', text='difference between a mcdouble and a double cheeseburger')
GenericQuery(query_id='148538', text='difference between rn and bsn')
GenericQuery(query_id='156493', text='do goldfish grow')
GenericQuery(query_id='168216', text='does legionella pneumophila cause pneumonia')
GenericQuery(query_id='182539', text='example of monotonic function')
GenericQuery(query_id='183378', text='exons definition biology')
GenericQuery(query_id='207786', text='how

In [17]:
pt.Experiment(
    [tf_idf, bm25, pl2, dph, linear],
    topic,
    qrels,
    ["P.5", "recall_5", "P.10", "recall_10", "map", "ndcg", "ndcg_cut_10", 'recip_rank'],
    ["TF-IDF", "BM25", "PL2", "DPH", "Combination of BM25 and DPH"]
)

,name,P.5,recall_5,P.10,recall_10,map,ndcg,ndcg_cut_10,recip_rank
0,TF-IDF,0.637209,0.076696,0.595349,0.122053,0.369486,0.593198,0.478310,0.795028
1,BM25,0.641860,0.077665,0.597674,0.122340,0.370004,0.593433,0.479540,0.795028
2,PL2,0.623256,0.073960,0.583721,0.120977,0.353712,0.581812,0.466875,0.788096
3,DPH,0.683721,0.081294,0.613953,0.127234,0.372805,0.590368,0.502513,0.799962
4,Combination of BM25 and DPH,0.665116,0.080096,0.618605,0.127790,0.381550,0.608907,0.508826,0.806753


In [26]:
query_1 = "right pelvic pain causes"
query_2 = "what is theraderm used for"
query_3 = "hydrogen is a liquid below what temperature"
query_4 = "example of monotonic function"
query_5 = "what is the most popular food in switzerland"
bonus_query = "How is instant coffee made"

In [30]:
result_1 = bm25.search(query_1)
top_100_result_1 = result_1[:100]
print(top_100_result_1)

   qid    docid    docno  rank      score                     query
0    1  1479542  1479542     0  38.213874  right pelvic pain causes
1    1  1051498  1051498     1  37.830403  right pelvic pain causes
2    1  1950269  1950269     2  36.995804  right pelvic pain causes
3    1  2268657  2268657     3  36.668262  right pelvic pain causes
4    1   973923   973923     4  35.129767  right pelvic pain causes
..  ..      ...      ...   ...        ...                       ...
95   1  8594274  8594274    95  29.298757  right pelvic pain causes
96   1  4029852  4029852    96  29.248266  right pelvic pain causes
97   1  3380602  3380602    97  29.232699  right pelvic pain causes
98   1  6672435  6672435    98  29.230158  right pelvic pain causes
99   1  2525903  2525903    99  29.221739  right pelvic pain causes

[100 rows x 6 columns]


In [31]:
result_2 = bm25.search(query_2)
top_100_result_2 = result_2[:100]
print(top_100_result_2)

  qid    docid    docno  rank      score                       query
0   1  8651775  8651775     0  25.624201  what is theraderm used for
1   1  8651776  8651776     1  25.253417  what is theraderm used for
2   1  8651772  8651772     2  24.501527  what is theraderm used for
3   1  8651771  8651771     3  23.454053  what is theraderm used for
4   1  8651770  8651770     4  22.492470  what is theraderm used for


In [32]:
result_3 = bm25.search(query_3)
top_100_result_3 = result_3[:100]
print(top_100_result_3)

   qid    docid    docno  rank      score  \
0    1   128984   128984     0  32.949260   
1    1  2697752  2697752     1  32.917614   
2    1  7911557  7911557     2  32.891788   
3    1  4233385  4233385     3  30.612833   
4    1  4233383  4233383     4  30.439872   
..  ..      ...      ...   ...        ...   
95   1  4496820  4496820    95  25.274306   
96   1  5513583  5513583    96  25.274306   
97   1  6166230  6166230    97  25.253214   
98   1  3826656  3826656    98  25.120039   
99   1  1549775  1549775    99  25.084882   

                                          query  
0   hydrogen is a liquid below what temperature  
1   hydrogen is a liquid below what temperature  
2   hydrogen is a liquid below what temperature  
3   hydrogen is a liquid below what temperature  
4   hydrogen is a liquid below what temperature  
..                                          ...  
95  hydrogen is a liquid below what temperature  
96  hydrogen is a liquid below what temperature  
97  hydro

In [33]:
result_4 = bm25.search(query_1)
top_100_result_4 = result_4[:100]
print(top_100_result_4)

   qid    docid    docno  rank      score                     query
0    1  1479542  1479542     0  38.213874  right pelvic pain causes
1    1  1051498  1051498     1  37.830403  right pelvic pain causes
2    1  1950269  1950269     2  36.995804  right pelvic pain causes
3    1  2268657  2268657     3  36.668262  right pelvic pain causes
4    1   973923   973923     4  35.129767  right pelvic pain causes
..  ..      ...      ...   ...        ...                       ...
95   1  8594274  8594274    95  29.298757  right pelvic pain causes
96   1  4029852  4029852    96  29.248266  right pelvic pain causes
97   1  3380602  3380602    97  29.232699  right pelvic pain causes
98   1  6672435  6672435    98  29.230158  right pelvic pain causes
99   1  2525903  2525903    99  29.221739  right pelvic pain causes

[100 rows x 6 columns]


In [35]:
result_5 = bm25.search(query_5)
top_100_result_5 = result_5[:100]
print(top_100_result_5)

   qid    docid    docno  rank      score  \
0    1  3437288  3437288     0  31.084387   
1    1  6230227  6230227     1  28.894334   
2    1  6230226  6230226     2  28.036928   
3    1  6230229  6230229     3  27.835319   
4    1  1524406  1524406     4  27.260340   
..  ..      ...      ...   ...        ...   
95   1  5595063  5595063    95  19.387160   
96   1  8400012  8400012    96  19.387160   
97   1  1524648  1524648    97  19.358640   
98   1  1859403  1859403    98  19.250034   
99   1  2127850  2127850    99  19.203971   

                                           query  
0   what is the most popular food in switzerland  
1   what is the most popular food in switzerland  
2   what is the most popular food in switzerland  
3   what is the most popular food in switzerland  
4   what is the most popular food in switzerland  
..                                           ...  
95  what is the most popular food in switzerland  
96  what is the most popular food in switzerland  


In [36]:
bonus_result = bm25.search(bonus_query)
top_100_bonus_result = bonus_result[:100]
print(top_100_bonus_result)

   qid    docid    docno  rank      score                       query
0    1  7905153  7905153     0  36.707686  How is instant coffee made
1    1  7905160  7905160     1  36.400263  How is instant coffee made
2    1  5525763  5525763     2  35.967950  How is instant coffee made
3    1  2290647  2290647     3  35.418005  How is instant coffee made
4    1  5525769  5525769     4  34.900957  How is instant coffee made
..  ..      ...      ...   ...        ...                         ...
95   1  4788846  4788846    95  26.059555  How is instant coffee made
96   1  7041867  7041867    96  26.059555  How is instant coffee made
97   1  5531369  5531369    97  25.945074  How is instant coffee made
98   1   699562   699562    98  25.857849  How is instant coffee made
99   1  6760620  6760620    99  25.857849  How is instant coffee made

[100 rows x 6 columns]


In [37]:
docno_list_query_1 = list(top_100_result_1["docno"])
docno_list_query_2 = list(top_100_result_2["docno"])
docno_list_query_3 = list(top_100_result_3["docno"])
docno_list_query_4 = list(top_100_result_4["docno"])
docno_list_query_5 = list(top_100_result_5["docno"])
docno_list_bonus_query = list(top_100_bonus_result["docno"])
print(docno_list_query_1)

['1479542', '1051498', '1950269', '2268657', '973923', '2163701', '1310106', '1310107', '1310108', '5583224', '1310101', '3049941', '7113153', '8737051', '2959687', '8737053', '5629373', '7116533', '114148', '216492', '3980122', '6661536', '1418931', '678245', '7564977', '3980126', '8376906', '855969', '1538587', '1759357', '535375', '4387756', '7564980', '3075337', '6237430', '1950273', '855967', '6087671', '753265', '8594276', '1778458', '4918001', '1759360', '7718090', '5519177', '3778489', '4029846', '4387753', '7734612', '3728444', '4279451', '6237429', '6461840', '3012141', '3744940', '8594277', '2053095', '4917998', '465742', '907072', '6301568', '4918005', '8238825', '2016623', '5815950', '855964', '1310103', '5107817', '8376910', '5815958', '66437', '6665319', '1759354', '7138879', '7575368', '6327176', '5155014', '113238', '216490', '855968', '185675', '6753502', '2412300', '4901414', '4929952', '1897214', '6665318', '8420374', '8620356', '7358643', '6332633', '678250', '1225

In [38]:
doc_text_query_1 = []
doc_text_query_2 = []
doc_text_query_3 = []
doc_text_query_4 = []
doc_text_query_5 = []
doc_text_bonus_query = []

In [40]:
for docno in docno_list_query_1:
  doc = docstore.get(doc_id=docno)
  doc_text = doc.text
  doc_text_query_1.append(doc_text)

for docno in docno_list_query_2:
  doc = docstore.get(doc_id=docno)
  doc_text = doc.text
  doc_text_query_2.append(doc_text)

for docno in docno_list_query_3:
  doc = docstore.get(doc_id=docno)
  doc_text = doc.text
  doc_text_query_3.append(doc_text)

for docno in docno_list_query_4:
  doc = docstore.get(doc_id=docno)
  doc_text = doc.text
  doc_text_query_4.append(doc_text)

for docno in docno_list_query_5:
  doc = docstore.get(doc_id=docno)
  doc_text = doc.text
  doc_text_query_5.append(doc_text)

for docno in docno_list_bonus_query:
  doc = docstore.get(doc_id=docno)
  doc_text = doc.text
  doc_text_bonus_query.append(doc_text)

In [41]:
print(doc_text_query_1)

['LOWER Right Abdominal Pain. Lower right abdominal pain may be due to some of the same causes discussed under upper right abdominal pain. With the lower abdominal regions, the continuation with the pelvic cavity means that pathologies of the pelvic organs or structures may also be a cause of pain.', 'Lower right abdominal pain may be due to some of the same causes discussed under upper right abdominal pain. With the lower abdominal regions, the continuation with the pelvic cavity means that pathologies of the pelvic organs or structures may also be a cause of pain.', 'Chronic pelvic pain is pain in the area below your bellybutton and between your hips that lasts six months or longer. Chronic pelvic pain can have multiple causes. It can be a symptom of another disease, or it can be a condition in its own right. If your chronic pelvic pain appears to be caused by another medical problem, treating that problem may be enough to eliminate your pain.', "Some women develop serious complicati

In [42]:
bm25_qe = pt.BatchRetrieve(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "Bo1"})
bm25_qebo2 = pt.BatchRetrieve(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "Bo2"})
bm25_kl = pt.BatchRetrieve(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "KLComplete"})
bm25s_unstemmed = pt.BatchRetrieve(index, wmodel="BM25", properties={"termpipelines" : ""})

In [43]:
pt.Experiment(
    [bm25_qe, bm25s_unstemmed, bm25_qebo2, bm25_kl],
    topic,
    qrels,
    ["P.5", "recall_5", "P.10", "recall_10", "map", "ndcg", "ndcg_cut_10", 'recip_rank'],
    ["BM25 with QE Bo1", "BM25 with QE Bo2", "BM25 with QE KLComplete", "BM25 Unstemmed"]
)

,name,P.5,recall_5,P.10,recall_10,map,ndcg,ndcg_cut_10,recip_rank
0,BM25 with QE Bo1,0.655814,0.078538,0.632558,0.128515,0.399897,0.610858,0.508620,0.788124
1,BM25 with QE Bo2,0.641860,0.077665,0.597674,0.122340,0.370004,0.593433,0.479540,0.795028
2,BM25 with QE KLComplete,0.665116,0.079496,0.637209,0.129834,0.411071,0.615639,0.514531,0.801283
3,BM25 Unstemmed,0.655814,0.078669,0.616279,0.126123,0.385106,0.591519,0.499146,0.806498
